In [32]:
import pandas as pd
import numpy as np
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import string

In [33]:
df_original=pd.read_excel("../data/all_data_2023-07-30.xlsx")

In [34]:
df_original

,name,link,image,regular_price,category,store,date,discount_price
0,Notebook Acer Aspire A115-22 3020E 4GB/128GB/1...,https://www.anhoch.com/product/599268598/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,10580,Laptop,Anhoch,2023-07-30,NaN
1,Notebook Lenovo IP3 3020E 4GB/256GB SSD/Vega3/...,https://www.anhoch.com/product/599268488/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,11980,Laptop,Anhoch,2023-07-30,NaN
2,"Notebook HP 14 Chromebook N4000/4GB/64GB/14"" F...",https://www.anhoch.com/product/599872723/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,11980,Laptop,Anhoch,2023-07-30,NaN
3,Notebook Mediacom SmartBook Edge 15.6 Intel Du...,https://www.anhoch.com/product/599872559/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,12580,Laptop,Anhoch,2023-07-30,NaN
4,"Notebook Lenovo IP3 6305 4GB/128GB SSD/15.6"" F...",https://www.anhoch.com/product/599269541/noteb...,https://d1a68gwbwfmqto.cloudfront.net/img/prod...,12980,Laptop,Anhoch,2023-07-30,NaN
...,...,...,...,...,...,...,...,...
4282,GORENJE FH-301CW ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/29288320/go...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,22999,Freezers,TehnoMarket,2023-07-30,19992.0
4283,GORENJE FH-401CW ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/29288321/go...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,23999,Freezers,TehnoMarket,2023-07-30,20992.0
4284,INDESIT UI-6 1 W ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/740637/inde...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,27999,Freezers,TehnoMarket,2023-07-30,22992.0
4285,WHIRLPOOL WHE-3133.1 ЗАМРЗНУВАЧ,https://tehnomarket.com.mk/product/29289082/wh...,https://d3mrte3vpewnxc.cloudfront.net/img/prod...,28999,Freezers,TehnoMarket,2023-07-30,21992.0


In [35]:
df=df_original.copy()

In [36]:
def preprocess_text(text):
    text = text.lower()
    text = text.replace('\\', ' ').replace('/', ' ')
    text = ''.join([char for char in text if char not in string.punctuation])
    return text

In [37]:
df['name_preprocessed'] = df['name'].apply(preprocess_text)

In [38]:
nlp = spacy.load("..\\venv\Lib\site-packages\en_core_web_md\en_core_web_md-3.6.0")

In [40]:
def get_word_embeddings(row):
    doc = nlp(row['name_preprocessed'])
    return doc.vector

In [41]:
df['name_vectorized'] = df.apply(get_word_embeddings, axis=1)

In [42]:
df[['name_vectorized']]

,name_vectorized
0,"[-0.06938505, 0.17586645, -2.1457279, 1.868082..."
1,"[0.3066337, 0.7494219, -2.3564832, 1.2007939, ..."
2,"[-0.16489284, -0.42599443, -2.757652, 2.850247..."
3,"[-1.0704012, 0.25899056, -2.0284715, 1.422973,..."
4,"[0.0991978, 1.0053672, -2.5445256, 1.7200178, ..."
...,...
4282,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4283,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4284,"[-0.67712, -0.9719, 0.5978558, 1.8894199, 1.35..."
4285,"[-0.09362999, 0.31618333, -0.21285333, -0.674,..."


In [43]:
similarity_matrix = cosine_similarity(np.array(df['name_vectorized'].values.tolist()))

In [44]:
similarity_matrix

array([[ 1.0000002 ,  0.89559245,  0.8821745 , ...,  0.36691955,
        -0.0396841 , -0.0396841 ],
       [ 0.89559245,  0.9999999 ,  0.8302399 , ...,  0.31795424,
         0.00925287,  0.00925287],
       [ 0.8821745 ,  0.8302399 ,  1.0000001 , ...,  0.44009072,
        -0.02765227, -0.02765227],
       ...,
       [ 0.36691955,  0.31795424,  0.44009072, ...,  0.9999997 ,
         0.03756174,  0.03756174],
       [-0.0396841 ,  0.00925287, -0.02765227, ...,  0.03756174,
         1.        ,  1.        ],
       [-0.0396841 ,  0.00925287, -0.02765227, ...,  0.03756174,
         1.        ,  1.        ]], dtype=float32)

In [45]:
agg_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.01)d
cluster_labels = agg_clustering.fit_predict(similarity_matrix)

In [48]:
df['cluster_id']=cluster_labels

In [51]:
df[['name','cluster_id','name_vectorized']]

,name,cluster_id,name_vectorized
0,Notebook Acer Aspire A115-22 3020E 4GB/128GB/1...,1715,"[-0.06938505, 0.17586645, -2.1457279, 1.868082..."
1,Notebook Lenovo IP3 3020E 4GB/256GB SSD/Vega3/...,2270,"[0.3066337, 0.7494219, -2.3564832, 1.2007939, ..."
2,"Notebook HP 14 Chromebook N4000/4GB/64GB/14"" F...",2635,"[-0.16489284, -0.42599443, -2.757652, 2.850247..."
3,Notebook Mediacom SmartBook Edge 15.6 Intel Du...,2169,"[-1.0704012, 0.25899056, -2.0284715, 1.422973,..."
4,"Notebook Lenovo IP3 6305 4GB/128GB SSD/15.6"" F...",2208,"[0.0991978, 1.0053672, -2.5445256, 1.7200178, ..."
...,...,...,...
4282,GORENJE FH-301CW ЗАМРЗНУВАЧ,513,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4283,GORENJE FH-401CW ЗАМРЗНУВАЧ,513,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4284,INDESIT UI-6 1 W ЗАМРЗНУВАЧ,866,"[-0.67712, -0.9719, 0.5978558, 1.8894199, 1.35..."
4285,WHIRLPOOL WHE-3133.1 ЗАМРЗНУВАЧ,182,"[-0.09362999, 0.31618333, -0.21285333, -0.674,..."


In [46]:
product_clusters = {}
for cluster_id, product_name in zip(cluster_labels, df['name']):
    if cluster_id not in product_clusters:
        product_clusters[cluster_id] = []
    product_clusters[cluster_id].append(product_name)

In [47]:
for cluster_id, products in product_clusters.items():
    print(f"Cluster {cluster_id}:")
    for product in products:
        print(f"- {product}")

Cluster 1715:
- Notebook Acer Aspire A115-22 3020E 4GB/128GB/15.6" FullHD/36Wh/NoOS/Silver
Cluster 2270:
- Notebook Lenovo IP3 3020E 4GB/256GB SSD/Vega3/15.6" HD LED/DOS/UltraSlim/Grey
Cluster 2635:
- Notebook HP 14 Chromebook N4000/4GB/64GB/14" FHD/Chrome OS/Mouse/Sleeve
Cluster 2169:
- Notebook Mediacom SmartBook Edge 15.6 Intel Dual N3350/8GB/M2 SSD Slot/15.6" FullHD/BT/Silver
Cluster 2208:
- Notebook Lenovo IP3 6305 4GB/128GB SSD/15.6" FullHD/DOS/UltraSlim/Gray
Cluster 2578:
- Notebook Lenovo IP3 6305 4GB/128GB/15.6" FullHD AG 250N/DOS/ArcticGray
Cluster 1510:
- Notebook Mediacom SmartBook Edge 13 Intel Dual N3350/4GB/64GB SSD+M2 Slot/13.3" FHD/BT/Silver/Win10
Cluster 1749:
- Notebook Lenovo IP3 3020E 8GB/256GB/15.6" FullHD AG 250N/DOS/ArcticGray
Cluster 1806:
- Notebook Lenovo IP3 3020E 8GB/256GB SSD/Vega3/15.6" HD LED/DOS/UltraSlim/Grey
Cluster 2405:
- Notebook HP 250 G8 N4020 4GB/256GB/Intel UHD/15.6" HD/RJ45/USB-C/DOS
Cluster 2555:
- Notebook Lenovo IP3 6305 4GB/256GB/15.6" Ful